In [3]:
from scrapy import Selector
import requests
import json
import pandas as pd

In [ ]:
# Id :
# Adresse :
# Latitude :
# Longitude :

In [4]:

cookies = {
    '_gid': 'GA1.2.1009969029.1770488287',
    '_ga_PTH30TD9RH': 'GS2.1.s1770490921$o4$g0$t1770490921$j60$l0$h0',
    '_ga': 'GA1.2.1501809574.1770100097',
    '_ga_6J3L27WGQH': 'GS2.2.s1770490921$o4$g0$t1770490921$j60$l0$h0',
}

headers = {
    # 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36 OPR/126.0.0.0',
    # 'Cookie': '_gid=GA1.2.1009969029.1770488287; _ga_PTH30TD9RH=GS2.1.s1770490921$o4$g0$t1770490921$j60$l0$h0; _ga=GA1.2.1501809574.1770100097; _ga_6J3L27WGQH=GS2.2.s1770490921$o4$g0$t1770490921$j60$l0$h0',
}

response = requests.get('https://www.djezzy.dz/nos-boutiques/', headers=headers)

In [25]:
response.status_code

200

In [1]:
# print(response.text)

In [5]:
html_sel = Selector(text=response.text)
html_sel

<Selector query=None data='<html lang="fr" dir="ltr" class="ltr"...'>

## Option 1

In [ ]:
data_item = html_sel.css('div.google-maps::attr(data-items)').get()

# Print only the first characters of the result
data_item[:400]

'[{"cds":"ADRAR","code":"Z56","categorie":"C","adresse":"Groupe 74, Pr\\u00e9s souk Dinar Tayeb, Adrar.","email":"service-clientele@djezzy.dz","horaires_ouverture":"08H00-18H00","vendredi":"Ferm\\u00e9","fetes":"Ouvert","code_ouverture":"","latitude":"27.87194","longitude":"-0.28569","comment":""},{"cds":"Ain Defla - Emir Abdelkader","code":"Z53","categorie":"B","adresse":"Boulevard Emir Abdelkader, '

In [ ]:
items = json.loads(data_item)
len(items), items[:3]

(128,
 [{'cds': 'ADRAR',
   'code': 'Z56',
   'categorie': 'C',
   'adresse': 'Groupe 74, Prés souk Dinar Tayeb, Adrar.',
   'email': 'service-clientele@djezzy.dz',
   'horaires_ouverture': '08H00-18H00',
   'vendredi': 'Fermé',
   'fetes': 'Ouvert',
   'code_ouverture': '',
   'latitude': '27.87194',
   'longitude': '-0.28569',
   'comment': ''},
  {'cds': 'Ain Defla - Emir Abdelkader',
   'code': 'Z53',
   'categorie': 'B',
   'adresse': 'Boulevard Emir Abdelkader, Ain Defla.',
   'email': 'service-clientele@djezzy.dz',
   'horaires_ouverture': '08H00-18H00',
   'vendredi': 'Fermée',
   'fetes': 'Ouvert',
   'code_ouverture': '',
   'latitude': '36.26649',
   'longitude': '1.97418',
   'comment': ''},
  {'cds': 'Ain Defla - Khemis Meliana',
   'code': 'Z89',
   'categorie': 'C',
   'adresse': 'Rue Belsaadi Abdelkader, Khemis Meliana, Aïn Defla.',
   'email': 'service-clientele@djezzy.dz',
   'horaires_ouverture': '08H00-19H00',
   'vendredi': 'Fermée',
   'fetes': 'Ouvert',
   'code_

In [12]:
for item in items[:3]:
    print('CDS:', item['cds'])
    print('Adresse:', item['adresse'])
    print('Latitude:', item['latitude'])
    print('Longitude:', item['longitude'])
    print('---')


CDS: ADRAR
Adresse: Groupe 74, Prés souk Dinar Tayeb, Adrar.
Latitude: 27.87194
Longitude: -0.28569
---
CDS: Ain Defla - Emir Abdelkader
Adresse: Boulevard Emir Abdelkader, Ain Defla.
Latitude: 36.26649
Longitude: 1.97418
---
CDS: Ain Defla - Khemis Meliana
Adresse: Rue Belsaadi Abdelkader, Khemis Meliana, Aïn Defla.
Latitude: 36.26118
Longitude: 2.21461
---


In [37]:
pd.DataFrame(items).to_excel('locations.xlsx', index=False)

## Option 2

In [13]:
div_items2 = html_sel.css('div.js-cds-items.items a').getall()

In [15]:
item = div_items2[0]
print(item)

<a data-markerid="0" href="#" class="item marker-link">
                            <h3>ADRAR</h3>
                            <strong>Adresse</strong>
                            <address>
                                Groupe 74, Prés souk Dinar Tayeb, Adrar.                            </address>
                        </a>


In [16]:
item_sel = Selector(text=item)
item_sel

<Selector query=None data='<html><body><a data-markerid="0" href...'>

In [17]:
item_sel = Selector(text=item)
cds = item_sel.css('h3::text').get().strip()
adresse = item_sel.css('address::text').get().strip()

print(cds)
print(adresse)

ADRAR
Groupe 74, Prés souk Dinar Tayeb, Adrar.


In [18]:
final_data = []

for item in div_items2:
    item_sel = Selector(text=item)
    cds = item_sel.css('h3::text').get().strip()
    adresse = item_sel.css('address::text').get().strip()
    final_data.append({'cds': cds, 'adresse': adresse})



In [53]:
pd.DataFrame(final_data).to_excel('locations2.xlsx', index=False)